# Conenct to Google Drive

If the dataset is in your gogle Drive run the following comand, click on the link at the outuput and insert you authorization code. Then, your Dive folder will be Mout in the runtime files.


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

## Imports

In [1]:
import tensorflow as tf


import numpy as np
from IPython.display import HTML, display

import os
from os import listdir, mkdir, path

import io
import pandas as pd
import numpy as np
import random

from PIL import Image
from collections import namedtuple, OrderedDict

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

Auxiliar functions based on the dataset_util.py script  of the Tensorflow Object detection API 

In [2]:
def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def bytes_list_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


def float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def float_list_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
  

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 50%'
        >
         {value}
        </progress>
    """.format(value=value, max=max))

In [73]:
import cv2

def create_tf_example(file, images_path, annotations_path, labels_dict):
  
    car = file.split("_")[0] 
    if car == "tesla":
        car_folder = "model3"
    elif car == "lexus":
        car_folder = "gsf"
    else:
        car_folder = car

    sub_folder = file.split("_")[1] 
    if sub_folder == 'test':
        sub_folder = 'test_with_labels'
    image_filename = file.split(".")[0] +".png"

    image_file_path = images_path + "/{}/{}/grayscale_wholeImage/{}".format(car_folder,sub_folder,image_filename)

    with tf.io.gfile.GFile(image_file_path, 'rb') as fid:
        encoded_png = fid.read()
    encoded_png_io = io.BytesIO(encoded_png)
    image = Image.open(encoded_png_io)

    
    im = Image.open(image_file_path, 'r').convert('L')
    print('Pillow: ', image.mode, image.size)
    print('Pillow: ', im.mode, im.size)
    print(image)
    print(im)
        
    
    img = cv2.imread(image_file_path, cv2.IMREAD_UNCHANGED)
    print('OpenCV: ', img.shape)

    width, height = image.size

    filename = image_filename.encode('utf8')
    image_format = b'png'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    bbox_path = annotations_path + "/{}/{}/boundingBoxes_wholeImage/{}".format(car_folder,sub_folder,file)
    with open(bbox_path) as f:
        boxes = f.read().split("\n")

    for box in boxes:
        info = box.split(",")
        if len(info) == 5:
            # print(info)
            classes.append(int(info[0]))
            classes_text.append(labels_dict[info[0]].encode('utf8'))
            print(classes)
            print(classes_text)
            xmin = int(info[1]) / width
            ymin = int(info[2]) / height
            xmax = int(info[3]) / width
            ymax = int(info[4]) / height
            if xmin > xmax:
                print("erro xmin > xmax")
            xmins.append(int(info[1]) / width)
            ymins.append(int(info[2]) / height)
            xmaxs.append(int(info[3]) / width)
            ymaxs.append(int(info[4]) / height)
        
  
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': int64_feature(height),
        'image/width': int64_feature(width),
        'image/filename': bytes_feature(filename),
        'image/source_id': bytes_feature(filename),
        'image/encoded': bytes_feature(encoded_png),
        'image/format': bytes_feature(image_format),
        'image/object/bbox/xmin': float_list_feature(xmins),
        'image/object/bbox/xmax': float_list_feature(xmaxs),
        'image/object/bbox/ymin': float_list_feature(ymins),
        'image/object/bbox/ymax': float_list_feature(ymaxs),
        'image/object/class/text': bytes_list_feature(classes_text),
        'image/object/class/label': int64_list_feature(classes),
    }))
    return tf_example


def write_tf_record(files, record_path, max_files):
    with tf.io.TFRecordWriter(record_path) as writer:
        count = 0
        number_of_files = len(files)

        out_display = display(progress(0, 100), display_id=True)

        print("Writing: {}".format(record_name))
        for file in files[:max_files]:
            example = create_tf_example(file, input_images_path,
                                      input_bbox_annotations_path,
                                      labels_map_dict)
            writer.write(example.SerializeToString())
            count += 1
            out_display.update(progress(int((count/max_files)*100), 100))
        writer.close()


Label map definition for the SVIRO classes

In [4]:
labels_map_dict  = {
    "1" : "infant_seat",
    "2" : "child_seat",
    "3" : "person",
    "4" : "everyday_object"
}

Point the path to the location of the SVIRO dataset folder


In [5]:
# sviro_dataset_path = "/content/gdrive/MyDrive/Colab Notebooks/SVIRO_Challenge/dataset"
sviro_dataset_path = "C:/Users/ferwi/Google Drive/Colab Notebooks/SVIRO_Challenge/dataset"

In [6]:
input_images_path = sviro_dataset_path + "/grayscale"
input_bbox_annotations_path = sviro_dataset_path + "/boundingBoxes"
output_tf_record_output_path = sviro_dataset_path + "/tfrecords"

Listo of the cars folders, according to the SVIRO dataset folder name, that will be used for the training dataset.

In [7]:
cars_for_train = ["zoe", "hilux", "model3"]
cars_for_test = ["zoe", "hilux", "model3", "x5", "tiguan"]

training_split = 0.75

Get the train and test files with the bouding boxes annotations from the dataset

In [8]:
# get the train files with the bouding boxes annotations from the dataset
train_files = [] 
valid_files = [] 

random.seed(0)
for car in cars_for_train:
    print("Getting training files from: {}".format(car))
    car_train_dir = input_bbox_annotations_path + "/{}/train/boundingBoxes_wholeImage".format(car)
    files = [f for f in listdir(car_train_dir) if path.isfile(path.join(car_train_dir, f)) and f.endswith(".txt")]
    # shuffle the files and split them for concatenate in the test finles and validation files
    random.shuffle(files)
    split_point = int((len(files)+1)*training_split)
    train_files = train_files + files[:split_point] 
    valid_files = valid_files + files[split_point:] 

# get the test files with the bouding boxes annotations from the dataset
test_files = []
for car in cars_for_test:
    print("Getting test files from: {}".format(car))
    car_test_dir = input_bbox_annotations_path + "/{}/test_with_labels/boundingBoxes_wholeImage".format(car)
    test_files = test_files + [f for f in listdir(car_test_dir) if path.isfile(path.join(car_test_dir, f)) and f.endswith(".txt")]

random.shuffle(train_files)
random.shuffle(valid_files)
random.shuffle(test_files)

print("Total files for training: {}".format(len(train_files)))
print("Total files for validation: {}".format(len(valid_files)))
print("Total files for testing: {}".format(len(test_files)))

Getting training files from: zoe
Getting training files from: hilux
Getting training files from: model3
Getting test files from: zoe
Getting test files from: hilux
Getting test files from: model3
Getting test files from: x5
Getting test files from: tiguan
Total files for training: 4500
Total files for validation: 1500
Total files for testing: 2500


In [74]:
record_name = '/SVIRO_bla_images.tfrecord'
record_path = output_tf_record_output_path + record_name 

# write_tf_record(train_files, record_path, max_files=int(len(train_files)))
write_tf_record(train_files, record_path, max_files=3)

Writing: /SVIRO_bla_images.tfrecord
Pillow:  RGB (960, 640)
Pillow:  L (960, 640)
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=960x640 at 0x1C97181ED30>
<PIL.Image.Image image mode=L size=960x640 at 0x1C97181EFA0>
OpenCV:  (640, 960, 3)
[3]
[b'person']
[3, 3]
[b'person', b'person']
Pillow:  RGB (960, 640)
Pillow:  L (960, 640)
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=960x640 at 0x1C97181ECA0>
<PIL.Image.Image image mode=L size=960x640 at 0x1C97154ED00>
OpenCV:  (640, 960, 3)
[3]
[b'person']
[3, 3]
[b'person', b'person']
[3, 3, 2]
[b'person', b'person', b'child_seat']
Pillow:  RGB (960, 640)
Pillow:  L (960, 640)
<PIL.PngImagePlugin.PngImageFile image mode=RGB size=960x640 at 0x1C970473C40>
<PIL.Image.Image image mode=L size=960x640 at 0x1C9704737F0>
OpenCV:  (640, 960, 3)
[3]
[b'person']
[3, 2]
[b'person', b'child_seat']
[3, 2, 2]
[b'person', b'child_seat', b'child_seat']


In [55]:
record_name = '/SVIRO_valid_images.tfrecord'
record_path = output_tf_record_output_path + record_name 

write_tf_record(valid_files, record_path, max_files=int(len(valid_files)))
# write_tf_record(valid_files, record_path, max_files=25)

Writing: /SVIRO_valid_images.tfrecord


In [52]:
record_name = '/SVIRO_short_test_images.tfrecord'
record_path = output_tf_record_output_path + record_name 

# write_tf_record(test_files, record_path, max_files=int(len(test_files)))
write_tf_record(test_files, record_path, max_files=10)

Writing: /SVIRO_short_test_images.tfrecord
['3', '0', '0', '418', '639']
['3', '534', '61', '841', '639']
['3', '0', '0', '393', '639']
['2', '587', '190', '891', '639']
['3', '656', '223', '938', '639']
['4', '572', '362', '696', '481']
['3', '100', '267', '340', '639']
['2', '102', '244', '401', '639']
['2', '134', '229', '416', '601']
['1', '564', '221', '931', '639']
['1', '0', '219', '380', '639']
['3', '113', '420', '301', '639']
['1', '0', '338', '362', '639']
['4', '389', '433', '565', '561']
['4', '590', '411', '675', '470']
['2', '123', '207', '400', '628']
['3', '0', '81', '451', '639']
['1', '581', '294', '959', '639']
['3', '100', '251', '358', '639']
['3', '535', '59', '959', '639']
['2', '127', '205', '401', '620']
['1', '66', '238', '394', '639']
['2', '557', '196', '789', '562']


In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/training/train'